In [1]:
import os
from os import path
import numpy as np
import copy
from astropy.io import fits
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
from astropy.visualization import make_lupton_rgb
import sys

In [2]:
# New pixels
pwd = '/Users/jimenagonzalez/research/DSPL/Simulations-Double-Source-Gravitational-Lensing/Data/Cutouts-coadd2/DES0229+0335/124779474/'
filename_g = 'DESJ022754.4277+035455.9080_g.fits'
filename_i = 'DESJ022754.4277+035455.9080_i.fits'
filename_r = 'DESJ022754.4277+035455.9080_r.fits'
with fits.open(pwd + filename_g) as hdul:
    data_g = hdul[0].data
with fits.open(pwd + filename_i) as hdul:
    data_i = hdul[0].data
with fits.open(pwd + filename_r) as hdul:
    data_r = hdul[0].data
data_g.shape

(46, 46)

In [3]:
def make_graphs(sim_i, sim_r, sim_g, cutout_i, cutout_r, cutout_g):
    loc = plticker.MultipleLocator(base=3.75)
    fig, (ax1, ax2, ax3) = plt.subplots(figsize=(15, 4), ncols=3)

    rgb = make_lupton_rgb(sim_i, sim_r, sim_g, Q=11., stretch=40.)
    ax1.set_title('Simulation')
    ax1.xaxis.set_ticklabels([])
    ax1.yaxis.set_ticklabels([])
    ax1.xaxis.set_major_locator(loc)
    ax1.yaxis.set_major_locator(loc)
    ax1.grid(which='major', axis='both', linestyle='-')
    original = ax1.imshow(rgb, aspect='equal')
    fig.colorbar(original, ax=ax1)

    rgb = make_lupton_rgb(cutout_i, cutout_r, cutout_g, Q=11., stretch=40.)
    ax2.set_title('Cutout')
    ax2.xaxis.set_ticklabels([])
    ax2.yaxis.set_ticklabels([])
    ax2.xaxis.set_major_locator(loc)
    ax2.yaxis.set_major_locator(loc)
    ax2.grid(which='major', axis='both', linestyle='-')
    cutout = ax2.imshow(rgb, aspect='equal')
    fig.colorbar(cutout, ax=ax2)
            
    rgb = make_lupton_rgb(sim_i+cutout_i, sim_r+cutout_r, sim_g+cutout_g, Q=11., stretch=40.)
    ax3.set_title('Complete')
    ax3.xaxis.set_ticklabels([])
    ax3.yaxis.set_ticklabels([])
    ax3.xaxis.set_major_locator(loc)
    ax3.yaxis.set_major_locator(loc)
    ax3.grid(which='major', axis='both', linestyle='-')
    projection = ax3.imshow(rgb, aspect='equal')
    fig.colorbar(projection, ax=ax3)
        
    plt.savefig('Data/Sim_complete/Image' + new_folder + '.png', bbox_inches='tight')
    plt.show() 

In [4]:
#Adding cutouts and simulations
directory_sim = '/Users/jimenagonzalez/research/DSPL/Simulations-Double-Source-Gravitational-Lensing/Data/Sim/'
directory_parent = 'Data/Cutouts-coadd2/'
folders = next(os.walk(directory_parent))[1]
# I should create the array & populate it
x = np.zeros((1,3,46,46)) #Complete Simulations
y = np.zeros((1,3,46,46)) #Cutouts
n = 0

#folder has as name an estimate of the coordinates
for folder in folders:
    new_folders = sorted(next(os.walk(directory_parent + folder))[1])
    #new_folder has as name the ID of each galaxy
    for new_folder in new_folders:
        #if(n >= 600):
        #    break
        path_sim_id = directory_sim + new_folder
        #check that the sim id folder exist (if not continue)
        if(os.path.exists(path_sim_id) == False):
            continue
        num_sim_id = int(len(os.listdir(path_sim_id))/3) # num sim with same lens
        #loop on each simulation produced using the same lens
        for j in range(1, num_sim_id+1):
            n = n + 1 #number of total simulations
            #loop over each file of the ID folder in cutouts
            files = sorted(next(os.walk(directory_parent + folder + '/' + new_folder))[2])
            for file in files:
                path_cutout = directory_parent + folder + '/' + new_folder + '/' + file
                path_sim = path_sim_id + '/' + str(j) + '_' + file[-6:]
                with fits.open(path_cutout) as hdul:
                    if file[-6:] == 'g.fits': cutout_g = hdul[0].data
                    if file[-6:] == 'i.fits': cutout_i = hdul[0].data
                    if file[-6:] == 'r.fits': cutout_r = hdul[0].data
                with fits.open(path_sim) as hdul:
                    if file[-6:] == 'g.fits': sim_g = hdul[0].data
                    if file[-6:] == 'i.fits': sim_i = hdul[0].data
                    if file[-6:] == 'r.fits': sim_r = hdul[0].data
            ele_sim = np.array([cutout_g + sim_g, cutout_r + sim_r, cutout_i + sim_i])
            x = np.append(x, [ele_sim], axis = 0)
            if(j == 1):
                ele_cut = np.array([cutout_g, cutout_r, cutout_i])
                y = np.append(y, [ele_cut], axis = 0)
            #make_graphs(sim_i, sim_r, sim_g, cutout_i, cutout_r, cutout_g)
x = np.delete(x, 0, axis = 0)
y = np.delete(y, 0, axis = 0)
print(len(x), len(y))

2271 1934


In [5]:
primary = fits.PrimaryHDU()
image = fits.ImageHDU(x, name="IMAGE")
hdu_list = fits.HDUList([primary, image])
hdu_list.writeto('Data/Sim_complete/complete.fits', overwrite=True)

In [6]:
primary = fits.PrimaryHDU()
image = fits.ImageHDU(y, name="IMAGE")
hdu_list = fits.HDUList([primary, image])
hdu_list.writeto('Data/Sim_complete/cutouts.fits', overwrite=True)

In [7]:
# convert -delay 140 -loop 0 *.png Complete_cut_sim.gif

In [8]:
hdu_list = fits.open('Data/Sim_complete/complete.fits')
my_images = hdu_list[1].data
print(type(my_images))
print(my_images.shape)

<class 'numpy.ndarray'>
(2271, 3, 46, 46)


In [9]:
hdu_list.close()

In [10]:
hdu_list = fits.open('Data/Sim_complete/cutouts.fits')
my_images = hdu_list[1].data
print(type(my_images))
print(my_images.shape)
hdu_list.close()

<class 'numpy.ndarray'>
(1934, 3, 46, 46)
